In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib import animation

### Punto fermo

In [2]:
# create a figure and axes
fig = plt.figure(figsize=(5,5))
ax1 = plt.subplot(1,1,1)

# set up the subplots as needed
ax1.set_xlim((0, 10))
ax1.set_ylim((0, 10))
ax1.set_xlabel('x')
ax1.set_ylabel('y')

# create objects that will change in the animation. These are
# initially empty, and will be given new values for each frame
# in the animation.
txt_title = ax1.set_title('')
pt1, = ax1.plot([], [], 'b.', ms=20)     # ax.plot returns a list of 2D line objects
plt.close()

In [3]:
# animation function. This is called sequentially
def disegnaframe(t):
    x0 = 5
    y0 = 5
    x = x0
    y = y0
    
    pt1.set_data(x, y)
    txt_title.set_text('Tempo = {0:4d} s'.format(t))
    return (pt1,pt1)

In [4]:
anim = animation.FuncAnimation(fig, disegnaframe, frames=100, interval=20, blit=True)

In [5]:
HTML(anim.to_html5_video())

### Punto in moto uniforme

In [6]:
# create a figure and axes
fig = plt.figure(figsize=(5,5))
ax1 = plt.subplot(1,1,1)

# set up the subplots as needed
ax1.set_xlim((0, 10))
ax1.set_ylim((0, 10))
ax1.set_xlabel('x')
ax1.set_ylabel('y')

# create objects that will change in the animation. These are
# initially empty, and will be given new values for each frame
# in the animation.
txt_title = ax1.set_title('')
pt1, = ax1.plot([], [], 'b.', ms=20)     # ax.plot returns a list of 2D line objects
plt.close()

In [7]:
def disegnaframe(t):
    x0 = 5
    y0 = 5
    vx = 0.05
    vy = 0
    
    x = x0 + t * vx
    y = y0 + t * vy
    
    pt1.set_data(x, y)
    txt_title.set_text('Tempo = {0:4d} s'.format(t))
    return (pt1,pt1)

In [8]:
anim = animation.FuncAnimation(fig, disegnaframe, frames=100, interval=20, blit=True)

In [9]:
HTML(anim.to_html5_video())

### Punto in moto accelerato, forza costante

## <font color='red'>caduta gravi, moto proiettili</font>  

In [10]:
# create a figure and axes
fig = plt.figure(figsize=(5,5))
ax1 = plt.subplot(1,1,1)

# set up the subplots as needed
ax1.set_xlim((0, 10))
ax1.set_ylim((0, 10))
ax1.set_xlabel('x')
ax1.set_ylabel('y')

# create objects that will change in the animation. These are
# initially empty, and will be given new values for each frame
# in the animation.
txt_title = ax1.set_title('')
pt1, = ax1.plot([], [], 'b.', ms=20)     # ax.plot returns a list of 2D line objects
plt.close()

In [11]:
def disegnaframe(t):
    m = 5  #kg
    Fx = 0.01 #N
    Fy = 0 #N
    x0 = 0 #m
    y0 = 5 #m
    v0x = 0.01 #m/s
    v0y = 0    #m/s
    
    ax = Fx/m
    ay = Fy/m
    x = x0 + v0x * t + 0.5 * ax * t**2
    y = y0 + v0y * t + 0.5 * ay * t**2
    
    pt1.set_data(x, y)
    
    txt_title.set_text('Tempo = {0:4d} s'.format(t))
    return (pt1,pt1)

In [12]:
anim = animation.FuncAnimation(fig, disegnaframe, frames=100, interval=20, blit=True)

In [13]:
HTML(anim.to_html5_video())

### Due punti, forza gravitazionale

In [14]:
# create a figure and axes
fig = plt.figure(figsize=(5,5))
ax1 = plt.subplot(1,1,1)

# set up the subplots as needed
ax1.set_xlim((0, 10))
ax1.set_ylim((0, 10))
ax1.set_xlabel('x')
ax1.set_ylabel('y')

# create objects that will change in the animation. These are
# initially empty, and will be given new values for each frame
# in the animation.
txt_title = ax1.set_title('')

pt1, = ax1.plot([], [], 'b.', ms=20)     # ax.plot returns a list of 2D line objects
pt2, = ax1.plot([], [], 'b.', ms=20)     # ax.plot returns a list of 2D line objects
plt.close()

## <font color='red'>mettere formula Fgrav, aggiungere conservazione energia, alcuni algoritmi non la conservano</font>  

In [15]:
def getAcc(pos):
    """
    Calculate the acceleration on each particle due to Newton's Law 
    pos  is an N x 3 matrix of positions
    mass is an N x 1 vector of masses
    G is Newton's Gravitational constant
    softening is the softening length
    a is N x 3 matrix of accelerations
    """
    
    N = pos.shape[0]
    a = np.zeros((N,2))
    softening = 0.1 # softening length
    G = 1.0         # Newton's Gravitational Constant
    global mass

    for i in range(N):
        for j in range(N):
            dx = pos[j,0] - pos[i,0];
            dy = pos[j,1] - pos[i,1];
            inv_r3 = (dx**2 + dy**2 + softening**2)**(-1.5);
            a[i,0] +=  G * (dx * inv_r3) * mass[j];
            a[i,1] +=  G * (dy * inv_r3) * mass[j];
            
    return a

# Simulation parameters
N  = 2    # Number of particles
dt = 0.01 # timestep

# Generate Initial Conditions
#np.random.seed(42) # set the random number generator seed
mass = 20.0*np.ones((N,1))/N  # total mass of particles is 20
pos  = np.random.normal(5,2,(N,2))#np.random.randn(N,2)+5 # randomly selected positions and velocities
vel  = np.random.normal(0,1,(N,2))#np.random.randn(N,2)

# calculate initial gravitational accelerations
acc = getAcc(pos)

def disegnaframe(t):
    global pos
    global vel
    global acc
    global dt
    
    # (1/2) kick
    vel += acc * dt/2.0

    # drift
    pos += vel * dt

    # update accelerations
    acc = getAcc(pos)

    # (1/2) kick
    vel += acc * dt/2.0

    pt1.set_data(pos[0,0], pos[0,1])
    pt2.set_data(pos[1,0], pos[1,1])

    txt_title.set_text('Tempo = {0:4d} s'.format(t))
    return (pt1,pt2)

In [16]:
anim = animation.FuncAnimation(fig, disegnaframe, frames=100, interval=20, blit=True)

In [17]:
HTML(anim.to_html5_video())

### N punti, forza gravitazionale

In [18]:
# create a figure and axes
fig = plt.figure(figsize=(5,5))
ax1 = plt.subplot(1,1,1)

# set up the subplots as needed
ax1.set_xlim((0, 10))
ax1.set_ylim((0, 10))
ax1.set_xlabel('x')
ax1.set_ylabel('y')

# create objects that will change in the animation. These are
# initially empty, and will be given new values for each frame
# in the animation.
txt_title = ax1.set_title('')

pts = []
N = 10 # Number of particles
for i in range(N):
    pts.append(ax1.plot([], [], 'b.', ms=20)) # ax.plot returns a list of 2D line objects
    plt.close()

In [28]:
def getAcc(pos):
    """
    Calculate the acceleration on each particle due to Newton's Law 
    pos  is an N x 3 matrix of positions
    mass is an N x 1 vector of masses
    G is Newton's Gravitational constant
    softening is the softening length
    a is N x 3 matrix of accelerations
    """
    
    N = pos.shape[0]
    a = np.zeros((N,2))
    softening = 0.001 # softening length
    G = 1.0         # Newton's Gravitational Constant
    global mass

    for i in range(N):
        for j in range(N):
            dx = pos[j,0] - pos[i,0];
            dy = pos[j,1] - pos[i,1];
            inv_r3 = (dx**2 + dy**2 + softening**2)**(-1.5);
            a[i,0] +=  G * (dx * inv_r3) * mass[j];
            a[i,1] +=  G * (dy * inv_r3) * mass[j];
            
    return a

# Simulation parameters
dt        = 0.01 # timestep

# Generate Initial Conditions
#np.random.seed(42) # set the random number generator seed
mass = 20.0*np.ones((N,1))/N  # total mass of particles is 20
pos  = np.random.normal(5,2,(N,2))#np.random.randn(N,2)+5 # randomly selected positions and velocities
vel  = np.random.normal(0,1,(N,2))#np.random.randn(N,2)

# calculate initial gravitational accelerations
acc = getAcc(pos)

def disegnaframe(t):
    global pos
    global vel
    global acc
    global dt
    
    # (1/2) kick
    vel += acc * dt/2.0

    # drift
    pos += vel * dt

    # update accelerations
    acc = getAcc(pos)

    # (1/2) kick
    vel += acc * dt/2.0

    for i in range(N):
        pts[i][0].set_data(pos[i,0], pos[i,1])

    txt_title.set_text('Tempo = {0:4d} s'.format(t))
    return (pts[0][0],pts[0][0])

In [29]:
anim = animation.FuncAnimation(fig, disegnaframe, frames=100, interval=20, blit=True)

In [30]:
HTML(anim.to_html5_video())

### N punti, forza gravitazionale, merging

In [102]:
# create a figure and axes
fig = plt.figure(figsize=(5,5))
ax1 = plt.subplot(1,1,1)

# set up the subplots as needed
ax1.set_xlim((0, 10))
ax1.set_ylim((0, 10))
ax1.set_xlabel('x')
ax1.set_ylabel('y')

# create objects that will change in the animation. These are
# initially empty, and will be given new values for each frame
# in the animation.
txt_title = ax1.set_title('')

#pts = []
N = 8 # Number of particles
#for i in range(N):
#    pts.append(ax1.plot([], [], 'b.', ms=20)) # ax.plot returns a list of 2D line objects
plt.close()

In [107]:
def getAcc(pos):
    """
    Calculate the acceleration on each particle due to Newton's Law 
    pos  is an N x 3 matrix of positions
    mass is an N x 1 vector of masses
    G is Newton's Gravitational constant
    softening is the softening length
    a is N x 3 matrix of accelerations
    """

    N = pos.shape[0]
    a = np.zeros((N,2))
    #softening = 0.001 # softening length
    G = 1.0         # Newton's Gravitational Constant
    global mass

    for i in range(N):
        for j in range(N):
            dx = pos[j,0] - pos[i,0]
            dy = pos[j,1] - pos[i,1]
            inv_r3 = (dx**2 + dy**2 + 0.000001**2)**(-1.5)
            a[i,0] +=  G * (dx * inv_r3) * mass[j]
            a[i,1] +=  G * (dy * inv_r3) * mass[j]

    return a

def merge():
    global N
    global pos
    global vel
    global acc
    global mass
    #global pts
    
    soglia_r = 0.1
    
    merge_indices = []
    #merging
    for i in range(N):
        for j in np.arange(i+1,N,1):
            dx = pos[j,0] - pos[i,0]
            dy = pos[j,1] - pos[i,1]
            if((dx**2 + dy**2) < soglia_r):
                merge_indices.append(i)
    #print(merge_indices)
    pos  = np.delete(pos,merge_indices,axis=0)
    vel  = np.delete(vel,merge_indices,axis=0)
    acc  = np.delete(acc,merge_indices,axis=0)
    mass = np.delete(mass,merge_indices,axis=0)
    #pts  = np.delete(pts,merge_indices,axis=0)
    N -= len(merge_indices)
    
# Simulation parameters
dt = 0.01 # timestep

# Generate Initial Conditions
#np.random.seed(42) # set the random number generator seed
mass = 20.0*np.ones((N,1))/N  # total mass of particles is 20
pos  = np.random.normal(5,2,(N,2))#np.random.randn(N,2)+5 # randomly selected positions and velocities
vel  = np.random.normal(0.1,1,(N,2))#np.random.randn(N,2)

#
merge()

# calculate initial gravitational accelerations
acc = getAcc(pos)

def disegnaframe(t):
    global pos
    global vel
    global acc
    global dt
    
    # (1/2) kick
    vel += acc * dt/2.0

    # drift
    pos += vel * dt
    
    #check for mergings
    merge()

    # update accelerations
    acc = getAcc(pos)

    # (1/2) kick
    vel += acc * dt/2.0

    pts = []
    for i in range(N):
        pts.append(ax1.plot(pos[i,0], pos[i,1], 'b.', ms=20))
        #pts[i][0].set_data()
    #print(N)
    #pts = ax1.plot(pos[:,0], pos[:,1], 'b.', ms=20)
    plt.close()

    txt_title.set_text('Tempo = {0:4d} s'.format(t))
    return (pts[0][0],pts[0][0])

In [108]:
anim = animation.FuncAnimation(fig, disegnaframe, frames=20, interval=20, blit=True)

In [109]:
HTML(anim.to_html5_video())

## <font color='red'>merging tra masse se r < soglia, distribuzione uniforme vs gaussiana, orbite aperte vs chiuse</font>  

In [50]:
def merge():
    global N
    global pos
    
    soglia_r = 0.000001
    
    merge_indices = []
    #merging
    for i in range(N):
        for j in np.arange(i+1,N,1):
            dx = pos[j,0] - pos[i,0]
            dy = pos[j,1] - pos[i,1]
            if((dx**2 + dy**2) < soglia_r):
                merge_indices.append(i)
    print(merge_indices)
    pos = np.delete(pos,merge_indices,axis=0)

N = 4
pos = np.array([[2,1],
                [3,4],
                [3,4],
                [5,6]])
merge()

print(pos)

[1]
[[2 1]
 [3 4]
 [5 6]]


# Da https://medium.com/swlh/create-your-own-n-body-simulation-with-python-f417234885e9